In [1]:
import torch
import torchvision
import cv2
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
import torch.nn.functional as F
import time
import RPi.GPIO as GPIO

#### inicializamos Pytorch y cargamos el modelo entrenado


In [2]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
model.load_state_dict(torch.load('modelo_entrenado.pth'))

<All keys matched successfully>

#### tranferimos el modelo desde la memoria del cpu hacia la Gpu


In [3]:
device = torch.device('cuda')
model = model.to(device)

#### Varios pasos y mucho blabla:
##### Convert from BGR to RGB
##### Convert from HWC layout to CHW layout
##### Normalize using same parameters as we did during training (our camera provides values in 0, 255 range and training loaded images in 0, 1 range so we need to scale by 255.0
##### Transfer the data from CPU memory to GPU memory
##### Add a batch dimension

In [4]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#### definimos la funcion de pre-procesado e iniciamos la camara con el deslizador de si esta bloqueado o no


In [5]:
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
pieza_slider = widgets.FloatSlider(description='hay pieza', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#display(widgets.HBox([image, pieza_slider]))

#### ejecucion de la red neuronal(sin gpio)


In [6]:
def update(change):
    global pieza_slider
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    
    prob_no_pieza = float(y.flatten()[0])
    pieza_slider.value = (1 - prob_no_pieza)
    
    time.sleep(0.001)

In [7]:
update({'new': camera.value})  # we call the function once to intialize
camera.observe(update, names='value') # this attaches the 'update' function to the 'value' traitlet of our camera

#### Gpio

In [23]:
GPIO.setmode(GPIO.BCM)    
GPIO.setup(18, GPIO.OUT, initial=GPIO.HIGH) # es el pin numero 12
valor_gpio = pieza_slider #1= pieza, 0 = no_pieza
def salida_robot(valor):
    global valor_gpio
    while True:
        if valor_gpio == 1:
            GPIO.output(18, GPIO.HIGH)
            print ("valor a 1, hay pieza")
        else:
            GPIO.output(18, GPIO.LOW)
            print ("valor a 0, no hay pieza")
        
        GPIO.cleanup()

#### Mostramos el Slider y la camara

In [8]:
camera_link.link()
display(widgets.HBox([image, pieza_slider]))

In [ ]:
camera.stop()